In [1]:
from __future__ import print_function
import numpy as np
import pandas as pd
import pystan

import psycopg2

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context('notebook', font_scale=1.2)

import matplotlib.style
matplotlib.style.use('ggplot')
%matplotlib inline

from IPython.display import display

In [2]:
import pymc3 as pm
import theano
import theano.tensor as T
from scipy.stats.stats import pearsonr

In [3]:
# create a database connection
sqluser = 'mimic'
dbname = 'mimic'
schema_name = 'mimiciii'

cur = None

In [4]:
if cur: 
    cur.close()
    con.close()

con = psycopg2.connect(dbname = dbname, user = sqluser)
cur = con.cursor()
cur.execute('SET search_path to ' + schema_name)

In [5]:
query = """
SELECT * FROM icu_features
"""
ef = pd.read_sql_query(query, con)
ef.head()

,icustay_id,hadm_id,subject_id,age,gender,height,weight,ethnicity,insurance,filter_vaso,...,pc_bronch,pc_cath,pc_echo,pc_pressor,pc_rhc,pc_thora,pc_vent,passed_filters,use_record,bmi
0,200001,152234,55973,22290 days 19:06:12,F,167.851667,27.669135,ASIAN - ASIAN INDIAN,Medicare,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,9.820741
1,200003,163557,27513,17625 days 19:50:04,M,177.800000,78.224998,WHITE,Private,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,24.744692
2,200006,189514,10950,19736 days 11:28:14,M,165.100000,82.400002,OTHER,Medicaid,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.229647
3,200007,129310,20707,15818 days 10:03:37,M,177.800000,126.000000,WHITE,Private,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.857223
4,200009,129607,29904,17353 days 10:34:32,F,160.020000,85.833331,WHITE,Private,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,33.520264


In [6]:
len(ef)

61532

In [7]:
F = (ef['use_record'] == 1)

In [8]:
ef = ef.loc[F]
len(ef)

9320

In [9]:
ef.ea_lv_systolic.value_counts()

 0.0    2297
-1.0     308
 3.0     220
 1.0     183
 2.0     147
-2.0      71
-3.0      30
Name: ea_lv_systolic, dtype: int64

## Calculate MDRD and add to feature table

In [10]:
def mdrd(data):
    creat = data.lab_creatinine
    age = data.age/np.timedelta64('1', 'Y')
    age.loc[age<0] = age.max()
    gender = data.gender.apply(lambda x: 0.742 if x == 'F' else 1)
    ethnicity = data.ethnicity.apply(lambda x: 1.212 if 'AFRICAN AMERICAN' in x else 1)
    return 175*creat**(-1.154) * age**(-0.203) * gender * ethnicity

In [11]:
value = mdrd(ef)
ef['mdrd'] = value
discrete = pd.cut(value, [0, 15, 30, 44, 60, 90, 1000000], labels=[5, 4, 3, 2, 1, 0], include_lowest=True)
ef['mdrd_discrete'] = discrete
discrete.value_counts()

0    2524
1    2068
2    1439
4    1280
3    1258
5     503
dtype: int64

In [12]:
discrete = pd.cut(value, [0, 30, 60, 1000000], labels=[2, 1, 0], include_lowest=True)
ef['mdrd_3'] = discrete
discrete.value_counts()

0    4592
1    2697
2    1783
dtype: int64

In [13]:
sum(np.isnan(ef.mdrd))

248

In [14]:
sum(np.isnan(ef.bmi))

3242

In [15]:
sum(np.isnan(ef.fb_day1_balance_ml))

637

In [16]:
sum(np.isnan(ef.fb_day2_balance_ml))

2041

In [17]:
sum(np.isnan(ef.fb_day3_balance_ml))

3832

## Create general features table

In [18]:
def normalize(x):
    x = x.copy()
    I = ~np.isnan(x)
    x[I] = (x[I] - x[I].mean())/x[I].std()
    return x

In [19]:
## is apache score everr missing?
sum(np.isnan(ef.apsiii))

0

In [20]:
X = pd.DataFrame()

# build the features table
age = ef.age/np.timedelta64('1', 'Y') # time in years
age_masked = age < 0 # if age is less than 0, then the person is above 89 (90?)
age = normalize(age)
age[age_masked] = 0
X['age'] = age
X['age_over_90'] = age_masked.astype(float)

gender = ef.gender
gender = (gender == 'F').astype(float)
X['gender'] = gender

bmi = ef.bmi
bmi_missing = np.isnan(bmi)
bmi = normalize(bmi)
bmi[bmi_missing] = 0
X['bmi'] = bmi
X['bmi_missing'] = bmi_missing.astype(float)

mech_vent = ef.vf_first_day_vent.astype(float)
X['mech_vent'] = mech_vent

chf = ef.ex_congestive_heart_failure.astype(float)
X['CHF'] = chf

mdrd = ef.mdrd
mdrd_missing = np.isnan(mdrd)
mdrd = normalize(mdrd)
mdrd[mdrd_missing] = 0
X['mdrd'] = mdrd

# MDRD missing is already largely covered by BMI missing
# the only other missing values come from missing creatinine lab
# so add that
creatinine_missing = np.isnan(ef.lab_creatinine)
X['creatinine_missing'] = creatinine_missing.astype(float)

apsiii = ef.apsiii
#apsiii_missing = np.isnan(apsiii)
apsiii = normalize(apsiii)
#apsiii[apsiii_missing] = 0
X['apsiii'] = apsiii
#X['apsiii_missing'] = apsiii_missing.astype(float)

no_echo = np.isnan(ef.ea_key)
#X['no_echo'] = no_echo.astype(float)

lvsys = ef.ea_lv_systolic
lv_missing = ((np.isnan(lvsys) | (lvsys < -1)) & ~no_echo).astype(float)
lv_normal = (lvsys == 0).astype(float)
lv_hyperdynamic = (lvsys == -1).astype(float)
lv_mild_depressed = (lvsys == 1).astype(float)
lv_mod_depressed = (lvsys == 2).astype(float)
lv_sev_depressed = (lvsys == 3).astype(float)
X['lv_missing'] = lv_missing
X['lv_normal'] = lv_normal
X['lv_hyper'] = lv_hyperdynamic
X['lv_mild'] = lv_mild_depressed
X['lv_moderate'] = lv_mod_depressed
X['lv_severe'] = lv_sev_depressed

tvpulm = ef.ea_tv_pulm_htn
tv_missing = ((np.isnan(tvpulm) | (tvpulm < -2)) & ~no_echo).astype(float)
tv_normal = (tvpulm == 0).astype(float)
tv_abnormal = ((tvpulm == -2) | (tvpulm > 0)).astype(float)
X['plm_htn_missing'] = tv_missing
X['plm_htn_normal'] = tv_normal
X['plm_htn_abnormal'] = tv_abnormal

rvcavity = ef.ea_rv_cavity
rvcav_missing = ((np.isnan(rvcavity) | (rvcavity < -2)) & ~no_echo).astype(float)
rvcav_normal = (rvcavity == 0).astype(float)
rvcav_small = (rvcavity == -1).astype(float)
rvcav_dilated = ((rvcavity == -2) | (rvcavity > 0)).astype(float)
X['rvcav_missing'] = rvcav_missing
X['rvcav_normal'] = rvcav_normal
X['rvcav_small'] = rvcav_small
X['rvcav_dilated'] = rvcav_dilated

lactate = ef.lab_lactate
lactate_missing = np.isnan(lactate)
lactate = normalize(lactate)
lactate[lactate_missing] = 0
X['lactate'] = lactate
X['lactate_missing'] = lactate_missing.astype(float)

## which ICU was treating the patient
sicu = ef.st_sicu.astype(float)
sicu[np.isnan(sicu)] = 0
nsicu = ef.st_nsicu.astype(float)
nsicu[np.isnan(nsicu)] = 0
micu = ef.st_micu.astype(float)
micu[np.isnan(micu)] = 0
X['sicu'] = sicu
X['nsicu'] = nsicu
X['micu'] = micu

X.head()

,age,age_over_90,gender,bmi,bmi_missing,mech_vent,CHF,mdrd,creatinine_missing,apsiii,...,plm_htn_abnormal,rvcav_missing,rvcav_normal,rvcav_small,rvcav_dilated,lactate,lactate_missing,sicu,nsicu,micu
1,0.100055,0.0,0.0,-0.002727,0.0,1.0,0.0,0.436365,0.0,-0.204515,...,1.0,0.0,1.0,0.0,0.0,0.716411,0.0,0.0,1.0,0.0
16,0.450966,0.0,1.0,0.000000,1.0,0.0,1.0,0.000000,1.0,0.364843,...,0.0,0.0,0.0,0.0,0.0,3.776100,0.0,0.0,0.0,1.0
20,0.164522,0.0,0.0,0.044261,0.0,0.0,0.0,0.404471,0.0,0.145859,...,0.0,0.0,1.0,0.0,0.0,0.065522,0.0,0.0,0.0,1.0
22,0.305922,0.0,0.0,-0.088287,0.0,0.0,0.0,0.729504,0.0,-1.036653,...,0.0,0.0,0.0,0.0,0.0,0.000000,1.0,1.0,0.0,0.0
25,0.390890,0.0,0.0,0.000000,1.0,0.0,0.0,-0.400352,0.0,2.204305,...,0.0,0.0,0.0,0.0,0.0,0.000000,1.0,1.0,0.0,1.0


## Features matrix with fluid balance

In [21]:
## make the features table. include fluid balance on days 1,2,3 of ICU stay
X_mortality = X.copy()

fluid_day1 = ef.fb_day1_balance_ml.values
fl_missing = np.isnan(fluid_day1) | ef.fb_day1_balance_truncated.eq(1).values # also filter insane fluid balances
fluid_day1[~fl_missing] = normalize(fluid_day1[~fl_missing])
fluid_day1[fl_missing] = 0
X_mortality['fluid_day1'] = fluid_day1
X_mortality['fluid_day1_missing'] = fl_missing.astype(float)

fluid_day2 = ef.fb_day2_balance_ml.values
fl_missing = np.isnan(fluid_day2) | ef.fb_day2_balance_truncated.eq(1).values
fluid_day2[~fl_missing] = normalize(fluid_day2[~fl_missing])
fluid_day2[fl_missing] = 0
X_mortality['fluid_day2'] = fluid_day2
X_mortality['fluid_day2_missing'] = fl_missing.astype(float)

fluid_day3 = ef.fb_day3_balance_ml.values
fl_missing = np.isnan(fluid_day3) | ef.fb_day3_balance_truncated.eq(1).values
fluid_day3[~fl_missing] = normalize(fluid_day3[~fl_missing])
fluid_day3[fl_missing] = 0
X_mortality['fluid_day3'] = fluid_day3
X_mortality['fluid_day3_missing'] = fl_missing.astype(float)

y_mortality = (ef.dod - ef.intime) < np.timedelta64(30, 'D')
y_mortality = y_mortality.astype(float)
X_mortality['mortality_day30'] = y_mortality

#select = ~((ef.dod - ef.intime) < np.timedelta64(3, 'D'))
#Xf = X_mortality.loc[select].drop('mortality_day30', axis = 1)
#yf = X_mortality.loc[select, 'mortality_day30']
Xf = X_mortality.drop('mortality_day30', axis = 1)
yf = X_mortality['mortality_day30']

In [22]:
Xf.shape

(9320, 34)

In [23]:
yf.mean(), yf.sum(), len(yf)

(0.2811158798283262, 2620.0, 9320)

## Logistic Regression model for predicting 30-day mortality

3-fold cross-validation to select regularization strength for L1 penalty with accuracy as target

In [24]:
import sklearn as sk

import sklearn.model_selection
import sklearn.linear_model
import sklearn.preprocessing

In [25]:
Xtr, Xte, ytr, yte = sk.model_selection.train_test_split(Xf, yf)

In [26]:
model = sk.linear_model.LogisticRegressionCV(cv = 3, penalty = 'l1', solver = 'liblinear')
model.fit(Xtr, ytr)
ypr = model.predict(Xte)

(ypr == yte).mean()

0.75493562231759659

In [27]:
pd.Series(model.coef_[0], index = list(Xtr.columns)).sort_values()

lv_normal            -0.167131
rvcav_normal         -0.131784
fluid_day1           -0.083444
gender               -0.072851
fluid_day3_missing   -0.065204
plm_htn_normal       -0.016074
lactate_missing      -0.000950
rvcav_dilated         0.000000
lv_mild               0.000000
lv_moderate           0.000000
sicu                  0.010416
CHF                   0.011951
fluid_day2            0.012738
rvcav_missing         0.029747
bmi                   0.059284
plm_htn_missing       0.066087
lv_missing            0.116678
mdrd                  0.118324
fluid_day1_missing    0.127234
fluid_day3            0.136822
lv_severe             0.148347
nsicu                 0.151066
fluid_day2_missing    0.222262
micu                  0.286401
bmi_missing           0.290106
plm_htn_abnormal      0.383830
lv_hyper              0.385965
mech_vent             0.422374
lactate               0.505210
apsiii                0.845745
rvcav_small           0.965531
age_over_90           1.104053
creatini

3-fold cross-validation to select regularization strength for L2 penalty with accuracy as target

In [28]:
model = sk.linear_model.LogisticRegressionCV(cv = 3, penalty = 'l2')
model.fit(Xtr, ytr)
ypr = model.predict(Xte)

(ypr == yte).mean()

0.75364806866952794

In [29]:
pd.Series(model.coef_[0], index = list(Xtr.columns)).sort_values()

lv_normal            -0.186288
plm_htn_normal       -0.091644
fluid_day1           -0.069201
rvcav_normal         -0.045313
gender               -0.040613
fluid_day3_missing   -0.034616
lv_mild              -0.021568
lv_moderate          -0.021055
plm_htn_missing      -0.016898
sicu                 -0.011632
lactate_missing       0.019507
fluid_day2            0.035158
bmi                   0.056076
rvcav_small           0.068352
mdrd                  0.076070
lv_severe             0.081821
nsicu                 0.081916
rvcav_missing         0.082692
rvcav_dilated         0.091744
CHF                   0.096973
lv_missing            0.101741
fluid_day3            0.112621
fluid_day1_missing    0.128855
fluid_day2_missing    0.208057
micu                  0.232699
lv_hyper              0.242823
bmi_missing           0.282517
plm_htn_abnormal      0.306017
mech_vent             0.361026
lactate               0.460021
age_over_90           0.577455
creatinine_missing    0.795151
apsiii  

## All interaction features

In [30]:
pf = sk.preprocessing.PolynomialFeatures(interaction_only = True)
X_p2 = pd.DataFrame(pf.fit_transform(Xf), 
    columns = pf.get_feature_names(list(Xf)))
X_p2.columns = [c.replace(' ', '*') for c in X_p2.columns]
X_p2.head()

,1,age,age_over_90,gender,bmi,bmi_missing,mech_vent,CHF,mdrd,creatinine_missing,...,fluid_day1_missing*fluid_day2,fluid_day1_missing*fluid_day2_missing,fluid_day1_missing*fluid_day3,fluid_day1_missing*fluid_day3_missing,fluid_day2*fluid_day2_missing,fluid_day2*fluid_day3,fluid_day2*fluid_day3_missing,fluid_day2_missing*fluid_day3,fluid_day2_missing*fluid_day3_missing,fluid_day3*fluid_day3_missing
0,1.0,0.100055,0.0,0.0,-0.002727,0.0,1.0,0.0,0.436365,0.0,...,-0.0,0.0,0.0,0.0,-0.0,-0.000000,-0.199688,0.0,0.0,0.0
1,1.0,0.450966,0.0,1.0,0.000000,1.0,0.0,1.0,0.000000,1.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,1.0,0.0
2,1.0,0.164522,0.0,0.0,0.044261,0.0,0.0,0.0,0.404471,0.0,...,-0.0,0.0,-0.0,0.0,-0.0,1.108478,-0.000000,-0.0,0.0,-0.0
3,1.0,0.305922,0.0,0.0,-0.088287,0.0,0.0,0.0,0.729504,0.0,...,-0.0,0.0,-0.0,0.0,-0.0,0.013806,-0.000000,-0.0,0.0,-0.0
4,1.0,0.390890,0.0,0.0,0.000000,1.0,0.0,0.0,-0.400352,0.0,...,-0.0,0.0,-0.0,0.0,-0.0,0.010561,-0.000000,-0.0,0.0,-0.0


L1

In [31]:
Xtr, Xte, ytr, yte = sk.model_selection.train_test_split(X_p2, yf)

In [32]:
model = sk.linear_model.LogisticRegressionCV(cv = 3, penalty = 'l1', solver = 'liblinear')
model.fit(Xtr, ytr)
ypr = model.predict(Xte)

(ypr == yte).mean()

0.77682403433476399

In [33]:
coefs = pd.Series(model.coef_[0], index = list(Xtr.columns)).sort_values()
coefs.head(n = 15)

1                                    -0.919716
mdrd*lv_moderate                     -0.583970
mech_vent*fluid_day1_missing         -0.580007
plm_htn_missing*fluid_day2_missing   -0.574535
plm_htn_missing*rvcav_dilated        -0.552757
fluid_day2_missing*fluid_day3        -0.548595
age_over_90*lactate                  -0.528906
micu*fluid_day3_missing              -0.509843
lv_missing*fluid_day1                -0.476980
fluid_day1                           -0.470833
apsiii*rvcav_missing                 -0.447268
age*lactate                          -0.376428
age*apsiii                           -0.374328
fluid_day1_missing*fluid_day2        -0.369221
fluid_day2                           -0.358981
dtype: float64

In [34]:
coefs.tail(n = 15)

lv_missing*fluid_day3                    0.389139
age*mech_vent                            0.397288
lv_severe*fluid_day2_missing             0.414278
mech_vent*fluid_day3_missing             0.416900
lactate                                  0.442054
apsiii*fluid_day3_missing                0.488437
creatinine_missing*fluid_day3_missing    0.559581
micu*fluid_day1_missing                  0.712437
creatinine_missing*fluid_day2_missing    0.823204
fluid_day3                               0.831327
age_over_90                              0.853124
apsiii                                   0.854135
fluid_day1*fluid_day2_missing            0.912607
fluid_day2*fluid_day3_missing            1.042561
age                                      1.830782
dtype: float64

In [35]:
(abs(coefs - 0) > 1e-6).value_counts()

False    380
True     216
dtype: int64

L2

In [36]:
model = sk.linear_model.LogisticRegressionCV(cv = 3, penalty = 'l2')
model.fit(Xtr, ytr)
ypr = model.predict(Xte)

(ypr == yte).mean()

0.77596566523605148

In [37]:
coefs = pd.Series(model.coef_[0], index = list(Xtr.columns)).sort_values()
coefs.head(n = 15)

micu*fluid_day3_missing               -0.357173
fluid_day1                            -0.317277
mech_vent*fluid_day1_missing          -0.316277
lv_missing*fluid_day1                 -0.262019
age_over_90*lactate                   -0.249532
plm_htn_missing*rvcav_dilated         -0.244698
apsiii*rvcav_missing                  -0.233580
plm_htn_missing*fluid_day2_missing    -0.230519
mdrd*lv_moderate                      -0.213372
rvcav_normal*fluid_day1               -0.197936
mech_vent*nsicu                       -0.194353
mech_vent*mdrd                        -0.189632
age*lactate                           -0.184225
plm_htn_abnormal*fluid_day3_missing   -0.177939
lv_severe*plm_htn_missing             -0.176366
dtype: float64

In [38]:
coefs.tail(n = 15)

age*fluid_day3                           0.276518
lactate                                  0.283114
micu*fluid_day1_missing                  0.298481
age_over_90                              0.325129
mech_vent*fluid_day3_missing             0.332590
creatinine_missing*fluid_day3_missing    0.338654
creatinine_missing*fluid_day2_missing    0.351025
age*micu                                 0.352583
age*mech_vent                            0.393238
fluid_day3                               0.441785
apsiii*fluid_day3_missing                0.457060
fluid_day1*fluid_day2_missing            0.539154
fluid_day2*fluid_day3_missing            0.578931
apsiii                                   0.605095
age                                      0.623228
dtype: float64

In [39]:
(abs(coefs - 0) > 1e-6).value_counts()

True     555
False     41
dtype: int64

## Add interaction features of interest

In [40]:
X_int = Xf.copy()

fluid_terms = [c for c in X_int.columns if 'fluid' in c]
echo_terms = [c for c in X_int.columns if ('lv_' in c) or ('plm_' in c) or ('rvcav_' in c)]
for fluid_term in fluid_terms: 
    X_int['{}:{}'.format(fluid_term, 'CHF')] = X_int[fluid_term] * X_int['CHF']
    X_int['{}:{}'.format(fluid_term, 'mech_vent')] = X_int[fluid_term] * X_int['mech_vent']
    
    for echo_term in echo_terms: 
        X_int['{}:{}'.format(fluid_term, echo_term)] = X_int[fluid_term] * X_int[echo_term]

In [41]:
Xtr, Xte, ytr, yte = sk.model_selection.train_test_split(X_int, yf)

L1

In [42]:
model = sk.linear_model.LogisticRegressionCV(cv = 3, penalty = 'l1', solver = 'liblinear')
model.fit(Xtr, ytr)
ypr = model.predict(Xte)

(ypr == yte).mean()

0.76437768240343351

In [43]:
coefs = pd.Series(model.coef_[0], index = list(Xtr.columns)).sort_values()
coefs.head(n = 15)

fluid_day1_missing:mech_vent          -0.904798
fluid_day3_missing                    -0.406156
fluid_day3:lv_mild                    -0.392339
fluid_day1:rvcav_normal               -0.344620
fluid_day1:lv_missing                 -0.306514
fluid_day3_missing:lv_normal          -0.291329
fluid_day2_missing:rvcav_normal       -0.225254
lv_mild                               -0.220220
fluid_day3_missing:plm_htn_abnormal   -0.214597
fluid_day1:plm_htn_abnormal           -0.172493
lv_normal                             -0.143264
fluid_day3:mech_vent                  -0.129779
gender                                -0.088202
fluid_day2:lv_mild                    -0.088037
fluid_day2:rvcav_normal               -0.078303
dtype: float64

In [44]:
coefs = pd.Series(model.coef_[0], index = list(Xtr.columns)).sort_values()
coefs.tail(n = 15)

bmi                                   0.202702
fluid_day2:plm_htn_abnormal           0.210522
micu                                  0.252405
bmi_missing                           0.288982
fluid_day3                            0.324760
fluid_day3_missing:plm_htn_missing    0.326552
fluid_day2_missing:mech_vent          0.482719
plm_htn_abnormal                      0.503250
fluid_day1_missing                    0.523688
lactate                               0.537959
fluid_day3_missing:mech_vent          0.657837
apsiii                                0.815875
age_over_90                           1.065196
creatinine_missing                    1.128229
age                                   1.808114
dtype: float64

L2

In [45]:
model = sk.linear_model.LogisticRegressionCV(cv = 3, penalty = 'l2', solver = 'liblinear')
model.fit(Xtr, ytr)
ypr = model.predict(Xte)

(ypr == yte).mean()

0.76609442060085842

In [46]:
coefs = pd.Series(model.coef_[0], index = list(Xtr.columns)).sort_values()
coefs.head(n = 15)

fluid_day1_missing:mech_vent          -0.433473
fluid_day3_missing                    -0.376779
fluid_day1:rvcav_normal               -0.271634
fluid_day1:lv_missing                 -0.211019
fluid_day3:lv_mild                    -0.206719
fluid_day3_missing:plm_htn_abnormal   -0.195583
fluid_day3_missing:lv_normal          -0.194625
fluid_day1:plm_htn_abnormal           -0.185765
fluid_day2:lv_mild                    -0.175622
plm_htn_normal                        -0.154969
fluid_day2_missing:rvcav_normal       -0.144387
plm_htn_missing                       -0.137875
fluid_day3:mech_vent                  -0.134899
lv_mild                               -0.134762
sicu                                  -0.117934
dtype: float64

In [47]:
coefs = pd.Series(model.coef_[0], index = list(Xtr.columns)).sort_values()
coefs.tail(n = 15)

fluid_day1_missing:CHF                0.154139
bmi                                   0.193078
lv_hyper                              0.198399
fluid_day3_missing:plm_htn_missing    0.221839
bmi_missing                           0.237872
fluid_day1_missing                    0.258294
fluid_day3                            0.262738
plm_htn_abnormal                      0.356503
fluid_day2_missing:mech_vent          0.360059
lactate                               0.508432
fluid_day3_missing:mech_vent          0.544719
age_over_90                           0.600132
creatinine_missing                    0.713437
apsiii                                0.780826
age                                   0.823726
dtype: float64